In [7]:
import xgboost
import shap
import pandas as pd

from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv("Life Expectancy Data Europe Recodificado.csv")
df.dropna(axis='rows', how='all', inplace=True) # Borra las filas que tengan todos los valores faltantes
df

Country  Year  Status  Life expectancy  Adult Mortality  Alcohol  \
0    Austria  2000       0             78.1               94    13.20   
1    Austria  2001       0             78.6               90    12.40   
2    Austria  2002       0             78.7               87    12.50   
3    Austria  2003       0             78.8               86    12.20   
4    Austria  2004       0             79.3               84    12.10   
..       ...   ...     ...              ...              ...      ...   
411   Sweden  2011       0             81.7               58     7.40   
412   Sweden  2012       0             81.7               56     7.40   
413   Sweden  2013       0             81.9               56     7.30   
414   Sweden  2014       0             82.3               55     7.30   
415   Sweden  2015       0             82.4               54     7.13   

     Percentage expenditure   BMI  Under-five deaths  Total expenditure  \
0                   2269.13  51.0                  0               1.60   
1                   2282.52  56.0                  0               1.12   
2                   2497.87  51.2                  0               1.27   
3                   3091.24  51.7                  0               1.48   
4                   3594.65  52.2                  0               1.56   
..                      ...   ...                ...                ...   
411                 6351.22  57.7                  0              11.70   
412                 6254.44  58.1                  0              11.80   
413                 6696.87  58.5                  0              11.97   
414                 6605.91  59.0                  0              11.93   
415                 5598.68  59.5                  0              10.80   

             GDP  Population  Thinness 1-19 years  Thinness 5-9 years  \
0    24517.26745      811566                  1.7                 1.9   
1    24489.73553      842293                  1.7                 1.9   
2    26351.37568      881957                  1.7                 1.9   
3     3212.93550     8121423                  1.7                 1.9   
4    36693.42620     8171966                  1.7                 1.9   
..           ...         ...                  ...                 ...   
411  59593.28711     9449213                  1.4                 1.3   
412  57134.77700     9519374                  1.4                 1.3   
413   6283.24522       96379                  1.4                 1.3   
414   5918.19898      969611                  1.5                 1.3   
415   5585.25847     9799186                  1.5                 1.4   

     Income composition of resources  Schooling  Average temperature  \
0                              0.833       15.4                 7.99   
1                              0.837       15.5                 7.04   
2                              0.847       16.1                 7.74   
3                              0.837       14.7                 7.37   
4                              0.841       14.7                 6.77   
..                               ...        ...                  ...   
411                            0.901       16.0                 4.01   
412                            0.903       15.8                 2.54   
413                            0.904       15.8                 3.37   
414                            0.906       15.8                 4.23   
415                            0.909       15.9                 3.97   

     CO2 Emissions  Cholesterol  Smoking  
0              7.9          3.9     52.3  
1              8.4          3.9      NaN  
2              8.6          3.9      NaN  
3              9.2          3.8      NaN  
4              9.3          3.8      NaN  
..             ...          ...      ...  
411            4.7          3.3      NaN  
412            4.4          3.3      NaN  
413            4.2          3.3      NaN  
414            4.0          3.3      NaN  
415         

In [9]:
Austria = df.loc[df['Country'] == 'Austria']
Belgium = df.loc[df['Country'] == 'Belgium']
Bulgaria = df.loc[df['Country'] == 'Bulgaria']
Croatia = df.loc[df['Country'] == 'Croatia']
Czechia = df.loc[df['Country'] == 'Czechia']
Denmark = df.loc[df['Country'] == 'Denmark']
Estonia = df.loc[df['Country'] == 'Estonia']
Finland = df.loc[df['Country'] == 'Finland']
France = df.loc[df['Country'] == 'France']
Germany = df.loc[df['Country'] == 'Germany']
Greece = df.loc[df['Country'] == 'Greece']
Hungary = df.loc[df['Country'] == 'Hungary']
Ireland = df.loc[df['Country'] == 'Ireland']
Italy = df.loc[df['Country'] == 'Italy']
Latvia = df.loc[df['Country'] == 'Latvia']
Lithuania = df.loc[df['Country'] == 'Lithuania']
Luxembourg = df.loc[df['Country'] == 'Luxembourg']
Malta = df.loc[df['Country'] == 'Malta']
Netherlands = df.loc[df['Country'] == 'Netherlands']
Poland = df.loc[df['Country'] == 'Poland']
Portugal = df.loc[df['Country'] == 'Portugal']
Romania = df.loc[df['Country'] == 'Romania']
Slovakia = df.loc[df['Country'] == 'Slovakia']
Slovenia = df.loc[df['Country'] == 'Slovenia']
Spain = df.loc[df['Country'] == 'Spain']
Sweden = df.loc[df['Country'] == 'Sweden']

In [10]:
def poblacion_recod(column):
    column['Poblacion Recod'] = (column['Population'] // 1000)
    return column

df2 = df.apply(poblacion_recod, axis='columns')
df2.head()

Country  Year  Status  Life expectancy  Adult Mortality  Alcohol  \
0  Austria  2000       0             78.1               94     13.2   
1  Austria  2001       0             78.6               90     12.4   
2  Austria  2002       0             78.7               87     12.5   
3  Austria  2003       0             78.8               86     12.2   
4  Austria  2004       0             79.3               84     12.1   

   Percentage expenditure   BMI  Under-five deaths  Total expenditure  ...  \
0                 2269.13  51.0                  0               1.60  ...   
1                 2282.52  56.0                  0               1.12  ...   
2                 2497.87  51.2                  0               1.27  ...   
3                 3091.24  51.7                  0               1.48  ...   
4                 3594.65  52.2                  0               1.56  ...   

   Population  Thinness 1-19 years  Thinness 5-9 years  \
0      811566                  1.7                 1.9   
1      842293                  1.7                 1.9   
2      881957                  1.7                 1.9   
3     8121423                  1.7                 1.9   
4     8171966                  1.7                 1.9   

   Income composition of resources  Schooling  Average temperature  \
0                            0.833       15.4                 7.99   
1                            0.837       15.5                 7.04   
2                            0.847       16.1                 7.74   
3                            0.837       14.7                 7.37   
4                            0.841       14.7                 6.77   

   CO2 Emissions  Cholesterol  Smoking  Poblacion Recod  
0            7.9          3.9     52.3              811  
1            8.4          3.9      NaN              842  
2            8.6          3.9      NaN              881  
3            9.2          3.8      NaN             8121  
4            9.3          3.8      NaN             8171  

[5 rows x 21 columns]

In [11]:
df3 = df.drop(df.columns[3], axis=1)
df3.head()

Country  Year  Status  Adult Mortality  Alcohol  Percentage expenditure  \
0  Austria  2000       0               94     13.2                 2269.13   
1  Austria  2001       0               90     12.4                 2282.52   
2  Austria  2002       0               87     12.5                 2497.87   
3  Austria  2003       0               86     12.2                 3091.24   
4  Austria  2004       0               84     12.1                 3594.65   

    BMI  Under-five deaths  Total expenditure          GDP  Population  \
0  51.0                  0               1.60  24517.26745      811566   
1  56.0                  0               1.12  24489.73553      842293   
2  51.2                  0               1.27  26351.37568      881957   
3  51.7                  0               1.48   3212.93550     8121423   
4  52.2                  0               1.56  36693.42620     8171966   

   Thinness 1-19 years  Thinness 5-9 years  Income composition of resources  \
0                  1.7                 1.9                            0.833   
1                  1.7                 1.9                            0.837   
2                  1.7                 1.9                            0.847   
3                  1.7                 1.9                            0.837   
4                  1.7                 1.9                            0.841   

   Schooling  Average temperature  CO2 Emissions  Cholesterol  Smoking  
0       15.4                 7.99            7.9          3.9     52.3  
1       15.5                 7.04            8.4          3.9      NaN  
2       16.1                 7.74            8.6          3.9      NaN  
3       14.7                 7.37            9.2          3.8      NaN  
4       14.7                 6.77            9.3          3.8      NaN

In [12]:
varis = list(df.columns)
varisAustria = list(Austria.columns)
target_var = varis.pop(2)
Country = varisAustria.pop(0)


xtrain, xtest, ytrain, ytest = train_test_split(df[varisAustria], df[target_var], random_state=1)
modelo = xgboost.XGBRegressor()
modelo.fit(xtrain, ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
shap.initjs()
explainer = shap.TreeExplainer(modelo)
shap_values = explainer.shap_values(xtest)
shap.summary_plot(shap_values, xtest)
shap.summary_plot(shap_values, xtest, plot_type='bar')

In [14]:
shap.plots.force(explainer.expected_value, shap_values)